# Problema das Moedas Correlacionadas

Dois amigos, João e José, estão jogando com duas moedas. Só que essas moedas são correlacionadas. Para a Moeda 1 há 30% de chance de que seja sorteado o valor “CARA”. Se para a Moeda 1 for sorteado o valor “CARA”, há 60% de chance de que seja sorteado o valor “CARA” para a Moeda 2. Ou seja, as moedas estão correlacionadas.

O jogo é o seguinte: Se após o lançamento das 2 moedas saírem dois valores “CARA” quem leva o dinheiro da aposta é o João. E se após o lançamento das 2 moedas saírem dois valores “COROA” quem leva o dinheiro da aposta é o José. Se saírem valores “CARA” e “COROA”, não há vencedores.

Você consegue fazer uma Simulação de Monte Carlo para determinar quem tem maior probabilidade de ganhar o jogo???

In [1]:
# Problema das moedas correlacionadas

import random

duplo_cara = 0
duplo_coroa = 0
cara_coroa = 0
amostra = 9000000

for i in range(amostra):
    # moeda1 = random.choice(['H','H','H','T','T','T','T','T','T','T'])
    moeda1 = 'H' if random.random() < 0.3 else 'T' # 30% de chances de sair cara (H)
    if moeda1 == 'H':
        # moeda2 = random.choice(['H','H','H','H','H','H','T','T','T','T'])
        moeda2 = 'H' if random.random() < 0.6 else 'T' # 60% de chances de sair cara (H)
    else:
        # moeda2 = random.choice(['H','H','H','H','H','H','T','T','T','T'])
        moeda2 = 'H' if random.random() < 0.3 else 'T' # 30% de chances de sair cara (H)
        # moeda2 = random.choice(['H','T'])
    
    if moeda1 + moeda2 == 'HH':
        duplo_cara += 1 # duplo_cara = duplo_cara + 1
    elif moeda1 + moeda2 == 'TT':
        duplo_coroa += 1 # duplo_coroa = duplo_coroa + 1
    else:
        cara_coroa += 1 # cara_coroa = cara_coroa + 1

prob_duplo_cara = duplo_cara / amostra * 100
prob_duplo_coroa = duplo_coroa / amostra * 100
prob_cara_coroa = cara_coroa / amostra * 100
total = prob_duplo_cara + prob_duplo_coroa + prob_cara_coroa

print('''Probabilidade de dar CARA + CARA: {0:.2f}%
Probabilidade de dar COROA + COROA: {1:.2f}%
Probabilidade de ninguem vencer: {2:.2f}%
Total: {3:.2f}%'''.format(prob_duplo_cara,prob_duplo_coroa,prob_cara_coroa,total)
)

Probabilidade de dar CARA + CARA: 18.00%
Probabilidade de dar COROA + COROA: 49.00%
Probabilidade de ninguem vencer: 32.99%
Total: 100.00%


# Problema da sazonalização

A carga que o consumidor quer contratar para os 12 meses do próximo ano é a carga acima. Ela foi “quantificada” tomando-se como base as cargas históricas de anos passados e também levando-se em consideração algumas ampliações fabris que vão acontecer no próximo ano.

Para a análise simplificada que pretendemos fazer aqui, vamos considerar que a carga a ser contratada é determinística. Ou seja, existe 100% de certeza que serão consumidos esses valores no próximo ano (o consumo não entra no problema como uma variável estocástica).

O PLD mensal que vamos considerar, também por razões de simplificação, é considerado determinístico e não estocástico. Ele foi “quantificado” tomando-se como base os PLDs históricos dos anos passados e também levando-se em consideração algumas suposições de comportamento futuro do PLD.

In [2]:
import numpy as np

consumo_mwmed = np.array([2.20, 3.52, 5.34, 2.70, 8.51, 2.92, 9.58, 12.4, 15.68, 13.95, 5.53, 5.53]) # em MWmed

numero_horas_mes = np.array([744.00, 672.00, 744.00, 720.00, 744.00, 720.00, 744.00, 744.00, 720.00, 744.00, 720.00, 744.00])

pld_reais_mwh = np.array([242.72, 165.98, 109.02, 132.63, 218.70, 336.99, 583.88, 583.88, 577.37, 249.36, 88.10, 66.67]) # em R$/MWh

In [3]:
meses = len(consumo_mwmed)

consumo_mwh = consumo_mwmed * numero_horas_mes # projecao_consumo_mwh

consumo_previsto_mwmed = np.sum(consumo_mwh) / np.sum(numero_horas_mes)

print('Meses: {}'.format(meses))
print('Consumo previsto em MW médio: {:,.2f}'.format(consumo_previsto_mwmed))

Meses: 12
Consumo previsto em MW médio: 7.36


**Pergunta-se:** Considerando-se os 4 Contratos disponíveis abaixo, qual é o melhor contrato para esse consumidor???

**CONTRATO-1** \
Quantidade Contrato (MWmed) = 8.0 \
Preço do Contrato (R$/MWh) = 120.0 \
Percentual de Sazo = 0.0 \
Percentual de Flex = 0.0


In [4]:

quantidade_contrato = 8
preco_contrato = 120
# quantidade_contrato = 15
# preco_contrato = 140
percentual_sazo = 0
percentual_flex = 0

# 1. Calcular a quantidade sazonalizada do contrato
contrato_mwh = quantidade_contrato * numero_horas_mes

# 2. Calcular a exposicao ao mcp:
# Subtratir o contrato sazonalizado do consumo sazonalizado previsto
# ATENÇÃO ao significado do sinal da conta: negativo é dívida ou crédito?!
exposicao_mcp_mwh = consumo_mwh - contrato_mwh # contrato_mwh - consumo_mwh

exposicao_mcp_reais = exposicao_mcp_mwh * pld_reais_mwh

total_exposicao_mcp_reais = np.sum(exposicao_mcp_reais) # * -1

# 3. Calcular o custo do contrato
# Multiplicar o contrato sazonalizado pelo valor negociado
custo_contrato_reais = contrato_mwh * preco_contrato

total_custo_contrato_reais = np.sum(custo_contrato_reais)

# 4. Somar o custo total do contrato com o total de exposição ao mcp
total_custo_contrato_exposicao_mcp_reais = total_custo_contrato_reais + total_exposicao_mcp_reais

print('Custo previsto o contrato: {:,.2f}'.format(total_custo_contrato_exposicao_mcp_reais))

# contrato_mwh = contratoMwh(quantidade_contrato,Numero_horas_mes)
# exposicao_mcp_mwh = expoMcpMwh(contrato_mwh,projecao_consumo_mwh)
# exposicao_mcp_reais = expoMcpReais(pld,quantidade_contrato,percentual_flex,projecao_consumo_mwh)
# custo_contrato = custoContrato(contrato_mwh,preco_contrato)
# contrato_mais_exposicao_mcp = contratoMaisExposicao(custo_contrato,exposicao_mcp_reais)
# print(contrato_mais_exposicao_mcp)
# sazo_minima , sazo_maxima = sazoMinimaMaxima(percentual_sazo)
# sazo_minima_mwh , sazo_maxima_mwh = sazoMinimaMaximaMwh(volume_flat_mwh,sazo_minima,sazo_maxima)
# sazo_contrato = sazoContrato(Numero_horas_mes,quantidade_contrato,percentual_sazo)
# sazo_flat_min_mwh , sazo_flat_max_mwh = sazoFlatMinMaxMwh(sazo_contrato,percentual_flex)
# expo_mcp_mwh = expoMcpMwh(sazo_contrato,projecao_consumo_mwh)
# total_exposicao_mcp_reais = expoMcpReais(expo_mcp_mwh,pld)
# custo_contrato = custoContrato(sazo_contrato,preco_contrato)
# custo_mais_exposicao = contratoMaisExposicao(custo_contrato,total_exposicao_mcp_reais)
# contratos.append(custo_mais_exposicao)
# graf_sazo(10,4,x,projecao_consumo_mwh,volume_flat_mwh,sazo_contrato,sazo_flat_min_mwh,sazo_flat_max_mwh,custo_mais_exposicao)


Custo previsto o contrato: 11,606,108.81



**CONTRATO-2** \
Quantidade Contrato (MWmed) = 6.0 \
Preço do Contrato (R$/MWh) = 190.0 \
Percentual de Sazo = 0.0 \
Percentual de Flex = 0.3


In [5]:
quantidade_contrato = 6.0
preco_contrato = 190.0
percentual_sazo = 0
percentual_flex = 0.3
# preco_contrato = 170.0
# percentual_flex = 0.4

# 1. Calcular a quantidade sazonalizada do contrato
contrato_mwh = quantidade_contrato * numero_horas_mes

# 2. Calcular a exposicao ao mcp:
# Subtratir o contrato sazonalizado do consumo sazonalizado previsto
# ATENÇÃO ao significado do sinal da conta: negativo é dívida ou crédito?!

# 2.1 Calcular limites máximo e minimo do percentual de flexibilidade sazonalizado
limite_maximo_flex_mwh = (1 + percentual_flex) * quantidade_contrato * numero_horas_mes
limite_minimo_flex_mwh = (1 - percentual_flex) * quantidade_contrato * numero_horas_mes

# 2.2 Calcular a quantidade de exposição se o consumo for acima ou abaixo dos limites
exposicao_mcp_mwh = np.zeros(meses)
for i in range(meses):
    if consumo_mwh[i] > limite_maximo_flex_mwh[i]:
        exposicao_mcp_mwh[i] = consumo_mwh[i] - limite_maximo_flex_mwh[i]
    if consumo_mwh[i] < limite_minimo_flex_mwh[i]:
        exposicao_mcp_mwh[i] = consumo_mwh[i] - limite_minimo_flex_mwh[i]

exposicao_mcp_reais = exposicao_mcp_mwh * pld_reais_mwh

total_exposicao_mcp_reais = np.sum(exposicao_mcp_reais) # * -1

# 3. Calcular o custo do contrato
# Multiplicar o contrato sazonalizado pelo valor negociado
custo_contrato_reais = np.zeros(meses)

for i in range(meses):
    # 3.1 Se o consumo estiver nos limites de flexbilidade, cobra o consumo pelo preço do contrato
    if limite_minimo_flex_mwh[i] <= consumo_mwh[i] <= limite_maximo_flex_mwh[i]:
        custo_contrato_reais[i] = consumo_mwh[i] * preco_contrato
    # 3.2 Se o consumo estiver acima do limite de flexbilidade, cobra o limite máximo pelo preço do contrato
    if consumo_mwh[i] > limite_maximo_flex_mwh[i]:
        custo_contrato_reais[i] = limite_maximo_flex_mwh[i] * preco_contrato
    # 3.1 Se o consumo estiver abaixo do limite de flexbilidade, cobra o limite mínimo pelo preço do contrato
    if consumo_mwh[i] < limite_minimo_flex_mwh[i]:
        custo_contrato_reais[i] = limite_minimo_flex_mwh[i] * preco_contrato

total_custo_contrato_reais = np.sum(custo_contrato_reais)

# 4. Somar o custo total do contrato com o total de exposição ao mcp
total_custo_contrato_exposicao_mcp_reais = total_custo_contrato_reais + total_exposicao_mcp_reais

print('Custo previsto o contrato: {:,.2f}'.format(total_custo_contrato_exposicao_mcp_reais))

# contrato_mwh = contratoMwh(quantidade_contrato,Numero_horas_mes)
# exposicao_mcp_mwh = consumoComFlex(quantidade_contrato,percentual_flex,projecao_consumo_mwh,meses) # expoMcpMwh(contrato_mwh,projecao_consumo_mwh)
# exposicao_mcp_reais = expoMcpReais(pld,quantidade_contrato,percentual_flex,projecao_consumo_mwh,meses)
# custo_contrato = custoContrato(contrato_mwh,preco_contrato)
# contrato_mais_exposicao_mcp = contratoMaisExposicao(custo_contrato,exposicao_mcp_reais)
# print(exposicao_mcp_mwh)

# sazo_minima , sazo_maxima = sazoMinimaMaxima(percentual_sazo)
# sazo_minima_mwh , sazo_maxima_mwh = sazoMinimaMaximaMwh(volume_flat_mwh,sazo_minima,sazo_maxima)
# sazo_contrato = sazoContrato(Numero_horas_mes,quantidade_contrato,percentual_sazo)
# sazo_flat_min_mwh , sazo_flat_max_mwh = sazoFlatMinMaxMwh(sazo_contrato,percentual_flex)
# expo_mcp_mwh = expoMcpMwh(sazo_contrato,projecao_consumo_mwh)
# total_exposicao_mcp_reais = expoMcpReais(expo_mcp_mwh,pld)
# custo_contrato = custoContrato(sazo_contrato,preco_contrato)
# custo_mais_exposicao = contratoMaisExposicao(custo_contrato,total_exposicao_mcp_reais)
# contratos.append(custo_mais_exposicao)
# graf_sazo(10,4,x,projecao_consumo_mwh,volume_flat_mwh,sazo_contrato,sazo_flat_min_mwh,sazo_flat_max_mwh,custo_mais_exposicao)


Custo previsto o contrato: 16,462,602.24



**CONTRATO-3** \
Quantidade Contrato (MWmed) = 5.0 \
Preço do Contrato (R$/MWh) = 220.0 \
Percentual de Sazo = 0.4 \
Percentual de Flex = 0.0


In [6]:
quantidade_contrato = 5
preco_contrato = 220
percentual_sazo = 0.4
percentual_flex = 0
# quantidade_contrato = 8
# percentual_sazo = 0.5

from scipy.optimize import linprog

# 1. Calcular a quantidade sazonalizada do contrato
contrato_mwh = quantidade_contrato * numero_horas_mes

# 2. Calcular a exposicao ao mcp:
# Subtratir o contrato sazonalizado do consumo sazonalizado previsto
# ATENÇÃO ao significado do sinal da conta: negativo é dívida ou crédito?!

# 2.1 Calcular limites máximo e minimo do percentual de flexibilidade sazonalizado
limite_maximo_flex_sazo_mwh = (1 + percentual_flex) * (1 + percentual_sazo) * quantidade_contrato * numero_horas_mes
limite_minimo_flex_sazo_mwh = (1 - percentual_flex) * (1 - percentual_sazo) * quantidade_contrato * numero_horas_mes

vetor_limites = []
for i in range(meses):
    limites = (limite_minimo_flex_sazo_mwh[i],limite_maximo_flex_sazo_mwh[i])
    vetor_limites.append(limites)

vetor_w = preco_contrato - pld_reais_mwh

matriz_igualdade_a = np.ones((1,12))

igualdade_b = quantidade_contrato * 365 * 24

solucao = linprog(vetor_w,A_ub=None,b_ub=None,A_eq=matriz_igualdade_a,b_eq=igualdade_b,bounds=vetor_limites,method='highs',callback=None,options=None,x0=None)

print('Valor ideal: ',round(solucao.fun, ndigits=2),
    '\nValores de X: ', solucao.x,
    '\nIterações realizadas: ', solucao.nit,
    '\nStatus: ', solucao.message,
)

# sazo_minima , sazo_maxima = sazoMinimaMaxima(percentual_sazo)
# sazo_minima_mwh , sazo_maxima_mwh = sazoMinimaMaximaMwh(volume_flat_mwh,sazo_minima,sazo_maxima)
# sazo_contrato = sazoContrato(Numero_horas_mes,quantidade_contrato,percentual_sazo)
# sazo_flat_min_mwh , sazo_flat_max_mwh = sazoFlatMinMaxMwh(sazo_contrato,percentual_flex)
# expo_mcp_mwh = expoMcpMwh(sazo_contrato,projecao_consumo_mwh)
# total_exposicao_mcp_reais = expoMcpReais(expo_mcp_mwh,pld)
# custo_contrato = custoContrato(sazo_contrato,preco_contrato)
# custo_mais_exposicao = contratoMaisExposicao(custo_contrato,total_exposicao_mcp_reais)
# contratos.append(custo_mais_exposicao)
# graf_sazo(10,4,x,projecao_consumo_mwh,volume_flat_mwh,sazo_contrato,sazo_flat_min_mwh,sazo_flat_max_mwh,custo_mais_exposicao)


Valor ideal:  -9487179.84 
Valores de X:  [8640. 2688. 2976. 2880. 2976. 8640. 8928. 8928. 8640. 8928. 2880. 2976.] 
Iterações realizadas:  1 
Status:  Optimization terminated successfully. (HiGHS Status 7: Optimal)



**CONTRATO-4** \
Quantidade Contrato (MWmed) = 8.0 \
Preço do Contrato (R$/MWh) = 240.0 \
Percentual de Sazo = 0.5 \
Percentual de Flex = 0.5

In [7]:
quantidade_contrato = 8
preco_contrato = 240
percentual_sazo = 0.5
percentual_flex = 0.5

# sazo_minima , sazo_maxima = sazoMinimaMaxima(percentual_sazo)
# sazo_minima_mwh , sazo_maxima_mwh = sazoMinimaMaximaMwh(volume_flat_mwh,sazo_minima,sazo_maxima)
# sazo_contrato = sazoContrato(Numero_horas_mes,quantidade_contrato,percentual_sazo)
# sazo_flat_min_mwh , sazo_flat_max_mwh = sazoFlatMinMaxMwh(sazo_contrato,percentual_flex)
# expo_mcp_mwh = expoMcpMwh(sazo_contrato,projecao_consumo_mwh)
# total_exposicao_mcp_reais = expoMcpReais(expo_mcp_mwh,pld)
# custo_contrato = custoContrato(sazo_contrato,preco_contrato)
# custo_mais_exposicao = contratoMaisExposicao(custo_contrato,total_exposicao_mcp_reais)
# contratos.append(custo_mais_exposicao)
# graf_sazo(10,4,x,projecao_consumo_mwh,volume_flat_mwh,sazo_contrato,sazo_flat_min_mwh,sazo_flat_max_mwh,custo_mais_exposicao)

# print(contratos)
# print('Melhor Contrato: CONTRATO ',melhorContrato(contratos))


## Errado

In [8]:
# import matplotlib.pyplot as plt

# def graf_sazo(largura,altura,meses,projecao,volume_flat,sazo_contrato,sazo_flat_min,sazo_flat_max,custo_contrato):
#     fig, ax = plt.subplots(figsize = (largura,altura))
#     ax.grid()
#     ax.plot(meses, projecao, color='navy', label='Projeção de consumo', **{'marker': '.'})
#     ax.plot(meses, volume_flat, color='green', label='Volume flat', **{'marker': '.'})
#     ax.plot(meses, sazo_contrato, color='firebrick', label='Proposta de sazo', **{'marker': '.'})
#     ax.fill_between(meses, sazo_flat_min, sazo_flat_max, alpha=.5, linewidth=0, label='Sazo + Flex')
#     ax.set_title('Sazonalização | Custo do contrato: R$ {0:.2f}'.format(custo_contrato))
#     ax.set_xticks(meses)
#     ax.set_xlabel('meses')
#     ax.set_ylabel('volume (MWh)')
#     ax.legend()
#     plt.show()
    
# x = np.arange(meses) + 1

# volume_flat_mwmed = np.sum(projecao_consumo_mwh) / np.sum(numero_horas_mes)
# volume_flat_mwh = volume_flat_mwmed * numero_horas_mes

# contratos = []

# def contratoMwh(contrato,Numero_horas_mes):
#     return contrato * Numero_horas_mes

# def expoMcpMwh(contrato,consumo):
#     return contrato - consumo

# def expoMcpReais(pld,quantidade_contrato,percentual_flex=0,projecao_consumo_mwh=0,meses=12):
#     if percentual_flex == 0:
#         return np.sum(expoMcpMwh(quantidade_contrato,projecao_consumo_mwh) * pld) * -1
#     else:
#         return consumoComFlex(quantidade_contrato,percentual_flex,projecao_consumo_mwh,meses)

# def custoContrato(contrato,preco,percentual_flex):
#     if percentual_flex == 0:
#         return np.sum(contrato * preco)
#     else:
        

# def contratoMaisExposicao(custoContratoReais,exposicaoMcpReais):
#     return custoContratoReais + exposicaoMcpReais

# # def sazoFlatMinMaxMwh(sazo_contrato,percentual_flex):
# #     sazoFlatMinMwh = sazo_contrato * (1 - percentual_flex)
# #     sazoFlatMaxMwh = sazo_contrato * (1 + percentual_flex)
# #     return sazoFlatMinMwh , sazoFlatMaxMwh

# def consumoComFlex(quantidade_contrato,percentual_flex,projecao_consumo_mwh,meses):
#     limiteMinimoFlex = quantidade_contrato * (1 - percentual_flex)
#     limiteMaximoFlex = quantidade_contrato * (1 + percentual_flex)
#     exposicao_com_flex = np.zeros(meses)
    
#     for i in range(meses):
#         if projecao_consumo_mwh[i] > limiteMaximoFlex:
#             exposicao_com_flex[i] = projecao_consumo_mwh[i] - limiteMaximoFlex
#         if projecao_consumo_mwh[i] < limiteMinimoFlex:
#             exposicao_com_flex[i] = projecao_consumo_mwh[i] - limiteMinimoFlex
    
#     return np.sum(exposicao_com_flex)

# def sazoMinimaMaxima(percentual_sazo):
#     sazoMinima = 1 - percentual_sazo
#     sazoMaxima = 1 + percentual_sazo
#     return sazoMinima , sazoMaxima 

# def sazoMinimaMaximaMwh(volume_flat_mwh,sazo_minima,sazo_maxima):
#     sazoMinimaMwh = volume_flat_mwh * sazo_minima
#     sazoMaximaMwh = volume_flat_mwh * sazo_maxima
#     return sazoMinimaMwh , sazoMaximaMwh

# def sazoContrato(Numero_horas_mes,quantidade_contrato,percentual_sazo):
#     return Numero_horas_mes * quantidade_contrato * (1 + percentual_sazo)


# def melhorContrato(contratos):
#     # contrato = 0
#     valor = contratos[-1]
#     for c,v in enumerate(contratos):
#         if v < valor:
#             valor = v
#             contrato = c
    
#     return contrato


# Linear Programming with Python
[Fonte](https://towardsdatascience.com/linear-programming-with-python-db7742b91cb)  
[Documentação](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html)

Um problema de otimização é composto por 03 componentes:
- **Função objetivo:** função matemática que se quer uma solução *maximizada* ou *minimizada*
- **Variáveis de decisão:** variáveis controladas que *influenciam* o modelo
- **Restrições:** conjunto de regras que as **variáveis de decisão** devem seguir

## Modelando o problema

1. **Definição do problema:** Descrever as variáveis de decisão, suas restrição e o objetivo da otimização (maximizar/minizar)
2. **Construção no modelo:** Representação matemática do problema, definição da função objetivo
3. **Solução do modelo:** Avalição de sensibilidade da solução encontrada com algumas alterações de parâmetros
4. **Validação do modelo:** Verifiacar se o modelo funciona como esperado
5. **Implementação:** Traduzir os resultados como recomendação de uma solução

## Programação linear (LP)

- Todos os objetivos são lineares
- Todos as restrições são lineares
- Todas as variáveis de decisão são contínuas

### Exemplo

min(z) = 10x' + 15x'' + 25x'''  
Restrições: {  
> 1x' + 1x'' + 1x''' >= 1000,  
> 1x' - 2x'' + 0x''' >= 0,  
> 0x' + 0x'' + 1x''' >= 340,  
> x', x'', x''' >= 0  

}

In [9]:
# matriz de restrições de desigualdades

A_ub = np.array([
    [-1,-1,-1], # 1x' + 1x'' + 1x''' >= 1000 --> -1x' - 1x'' - 1x''' <= -1000
    [-1, 2, 0], # 1x' - 2x'' + 0x''' >= 0    --> -1x' + 2x'' - 0x''' <= 0
    [ 0, 0,-1], # 0x' + 0x'' + 1x''' >= 340  --> -0x' - 0x'' - 1x''' <= -340
    [-1, 0, 0], # x' >= 0                    --> -x' <= 0
    [ 0,-1, 0], # x'' >= 0                   --> -x'' <= 0
    [ 0, 0,-1]  # x''' >= 0                  --> -x''' <= 0
])

# vetor de restrições de desigualdades

b_ub = np.array([-1000, 0,-340, 0, 0, 0])

# coeficientes da função objetivo

c = np.array([10,15,25])

resultado = linprog(c,A_ub=A_ub,b_ub=b_ub)

print('Valor ideal: ',round(resultado.fun, ndigits=2),
    '\nValores de X: ', resultado.x,
    '\nIterações realizadas: ', resultado.nit,
    '\nStatus: ', resultado.message,
)

Valor ideal:  15100.0 
Valores de X:  [660.   0. 340.] 
Iterações realizadas:  0 
Status:  Optimization terminated successfully. (HiGHS Status 7: Optimal)
